# Algoritmo de Cycle Cancelling

###### Nota: Algoritmo adaptado de https://github.com/scheng95/cs51finalproject/blob/master/to_submit/code/4cycle_cancel.py

## Importar librerías

In [14]:
import pandas as pd
import sys
import pprint
import time

## Creación de grafo

#### Matriz de capacidades

In [16]:
dfCaps = pd.read_csv("Matrices en CSV/capacidadescanciclos.csv", sep = ";", encoding = 'utf-8', header = None)
caps = dfCaps.to_numpy()
vertexNum = len(caps)

In [4]:
caps

array([[ 0, 50,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 25,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

#### Matriz de costos

In [17]:
dfCosts = pd.read_csv("Matrices en CSV/costoscanciclos.csv", sep = ";", encoding = 'utf-8', header = None)
costs = dfCosts.to_numpy()

In [18]:
costs

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.20142092, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Métodos útiles

### Encontrar un camino posible desde el nodo de origen al nodo de destino utilizando el algoritmo BFS

In [19]:
def bfs(source, dest, caps, resCaps, vertexNum):
  queue = [source]
  paths = {source: []}
  while queue:
    u = queue.pop(0)
    for v in range(vertexNum):
      if caps[u][v] != 0:
        if caps[u][v] - resCaps[u][v] > 0 and v not in paths:
          paths[v] = paths[u] + [(u, v)]
          if v == dest:
            # print("Possible path from ", source, " to ", dest, "->", paths[v])
            return paths[v]
          queue.append(v)
  return None

### Calcular el flujo máximo en el grafo con el algoritmo de Edmonds Karp. Dando como resultado el grafo residual en costos y capacidades

In [9]:
def edmondsKarp(source, dest, cap, cost, vertexNum):
  resCap = [[0] * vertexNum for _ in range(vertexNum)]
  resCost = [[0] * vertexNum for _ in range(vertexNum)]

  while True:
    path = bfs(source, dest, cap, resCap, vertexNum)
    if not path:
      break
        
    flow = min(cap[u][v] - resCap[u][v] for u, v in path)
    # print("Flow:\n", flow)
    
    for u, v in path:
      resCap[u][v] += flow
      resCost[u][v] = cost[u][v]
      resCap[v][u] -= flow
      resCost[v][u] = cost[u][v]*-1

  return (resCap, resCost)

### Encontrar los ciclos negativos en el grafo con el algoritmo Bellman Ford

In [10]:
def bellmanFord(graph, source, vertexNum):
  distances = [sys.maxsize for i in range(vertexNum)]
  prev = [-1 for i in range(vertexNum)]
  distances[source] = 0

  for i in range(vertexNum - 1):
    for u in range(vertexNum):
        for v in graph[u].keys():
          tempdist = sys.maxsize
          
          if distances[u] != sys.maxsize:
            tempdist = distances[u] + graph[u][v][1]
          if distances[v] > tempdist:
            distances[v] = tempdist
            prev[v] = u

  cycle = []
  for u in range(vertexNum):
    for v in graph[u].keys():
      if distances[v] > distances[u] + graph[u][v][1]:
        C = v
        for i in range(vertexNum):
          C = prev[C]
        v = C
        # temp = v
        while True:
          cycle.append(v)
          if v == C and len(cycle) > 1:
              break
          v = prev[v]
        cycle.reverse()
        final_cycle = []
        for index in range(len(cycle) - 1):
          final_cycle.append((cycle[index], cycle[index + 1]))
            
        # print("\nNegative cycle:\n", final_cycle)
        return final_cycle
        
  return None


### Método que implementa el algoritmo Cycle Cancelling

In [12]:
def cycleCancelling(source, dest, caps, costs, vertexNum):
    resCap, resCost = edmondsKarp(source, dest, caps, costs, vertexNum)

    # Poner el valor restante para cada arista
    for u in range(vertexNum):
      for v in range(len(caps)):
        if caps[u][v] != 0:
          if resCap[u][v] > 0:
            if u == dest + 1:
              resCap[u][v] = caps[u][v] - resCap[u][v]
              resCost[u][v] = 0
            else:
              resCap[u][v] = caps[u][v] - resCap[u][v]
              resCost[u][v] = costs[u][v]
                        

    # Añadir las aristas no utilizadas al grafo residual
    for u in range(vertexNum):
      for v in range(len(caps)):
        if caps[u][v] != 0:
          if resCap[v][u] == 0 and resCost[v][u] == 0:
            resCap[u][v] = caps[u][v]
            resCost[u][v] = costs[u][v]

    # Convertir las matrices residuales en una lista de diccionarios para leerlos desde el método de Bellman Ford
    resG = [{} for i in range(vertexNum)]
    for u in range(vertexNum):
      for v in range(vertexNum):
        if resCap[u][v] != 0 or resCost[u][v] != 0:
          resG[u][v] = (abs(resCap[u][v]), resCost[u][v])
    # print("\nResidual Graph: ")
    # pprint.pprint(resG)

    while True:
      cycle = bellmanFord(resG, dest, vertexNum)
      if not cycle:
        break
          
      flow = min(resG[u][v][0] for u, v in cycle)
      for u, v in cycle:
        temp = resG[u][v]
        if u in resG[v].keys():
          resG[v][u] = (resG[v][u][0] + flow, resG[v][u][1])
        else:
          resG[v][u] = (flow, -resG[u][v][1])
        if temp[0] == flow:
          del resG[u][v]
        else:
          resG[u][v] = (resG[u][v][0] - flow, resG[u][v][1])
          temp = resG[u][v]

    finalCost = 0
    # print("Final flow is: ")
    for u in range(vertexNum):
      for v in resG[u].keys():
        edge = resG[u][v]
        if edge[1] < 0:
          #print((v, u), ":", (edge[0], abs(edge[1])))
          print("-> Del nodo", v, "al nodo", u, "se envía un tráfico de", edge[0], "con un retardo de", abs(edge[1]))
          finalCost += (edge[0]) * (abs(edge[1]))

    return finalCost


## Ejecución

In [20]:
source = 0
dest = vertexNum - 1
print("Ruta óptima:")
#print("(Nodo del que sale, nodo al que llega) : (Tráfico que se envía, Retardo)")
finalCost = cycleCancelling(source, dest, caps, costs, vertexNum)
print("\nRetardo mínimo = ", finalCost)

Ruta óptima:
-> Del nodo 1 al nodo 5 se envía un tráfico de 9 con un retardo de 0.192245058
-> Del nodo 9 al nodo 8 se envía un tráfico de 6 con un retardo de 0.122244401
-> Del nodo 1 al nodo 8 se envía un tráfico de 2 con un retardo de 0.374331887
-> Del nodo 1 al nodo 9 se envía un tráfico de 39 con un retardo de 0.043189676
-> Del nodo 5 al nodo 13 se envía un tráfico de 9 con un retardo de 0.301797001
-> Del nodo 9 al nodo 13 se envía un tráfico de 17 con un retardo de 0.445052009
-> Del nodo 8 al nodo 13 se envía un tráfico de 8 con un retardo de 0.25593562
-> Del nodo 9 al nodo 14 se envía un tráfico de 16 con un retardo de 0.314895318
-> Del nodo 13 al nodo 18 se envía un tráfico de 21 con un retardo de 0.163697885
-> Del nodo 18 al nodo 19 se envía un tráfico de 8 con un retardo de 0.12868042
-> Del nodo 13 al nodo 22 se envía un tráfico de 13 con un retardo de 0.16257214
-> Del nodo 14 al nodo 22 se envía un tráfico de 16 con un retardo de 0.156101813
-> Del nodo 18 al nodo 2